In [1]:
# This is used to convert the xmls to a consolidated csv file for train and test data.
# The code will randomly split the training and testing data and consolidate the xmls for those into a single one.
# coding: utf-8

# # Object Detection Demo
# Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/installation.md) before you start.

# # Imports

# In[1]:

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

import glob
import pandas as pd
import xml.etree.ElementTree as ET


from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
# ## Object detection imports
# Here are the imports from the object detection module.

# In[3]:

from utils import label_map_util

from utils import visualization_utils as vis_util

from shutil import copyfile

from glob import glob

In [4]:
def xml_to_csv(path):
    print("path is:: " + path)
    xml_list = []
    for xml_file in glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


In [5]:
%cd images

/Users/kprayagai/Documents/Magesh/MachineLearning/iOS11ML/models-master/object_detection/images


In [9]:
g = glob('*.jpg')
shuf = np.random.permutation(g)

#print(shuf)
for i in range(150): 
    copyfile(shuf[i], 'train/' + shuf[i])
    #print(shuf[i])
    file_name = os.path.basename(shuf[i])
    #print(file_name)
    filename = os.path.splitext(file_name)[0] + '.xml'
    copyfile(filename, 'train/' + filename)

In [10]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(42): 
    copyfile(shuf[i], 'test/' + shuf[i])
    file_name = os.path.basename(shuf[i])
    filename = os.path.splitext(file_name)[0] + '.xml'
    copyfile(filename, 'test/' + filename)

In [11]:
for directory in ['train','test']:
        image_path = os.path.join(os.getcwd(), '{}'.format(directory))
        xml_df = xml_to_csv(image_path)
        #print(xml_df)
        xml_df.to_csv(os.path.join(os.getcwd(),'../data/{}_labels.csv'.format(directory)), index=None)
        print('Successfully converted xml to csv.')

path is:: /Users/kprayagai/Documents/Magesh/MachineLearning/iOS11ML/models-master/object_detection/images/train
Successfully converted xml to csv.
path is:: /Users/kprayagai/Documents/Magesh/MachineLearning/iOS11ML/models-master/object_detection/images/test
Successfully converted xml to csv.


In [ ]:
# ## Download Model

# In[5]:

# Commented out since model is downloaded already.
#opener = urllib.request.URLopener()
#opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
#tar_file = tarfile.open(MODEL_FILE)
#for file in tar_file.getmembers():
 #   file_name = os.path.basename(file.name)
 #   if 'frozen_inference_graph.pb' in file_name:
 #   tar_file.extract(file, os.getcwd())


In [ ]:
# ## Load a (frozen) Tensorflow model into memory.
# In[6]:

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
